## Covid19 Dashboard

In [5]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [6]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## PHE Data
This data comes from the Public Health England database. 
It contains the following pieces of information:
New Cases By Specimen Date
Cumulative Cases By Specimen Date
New Cases PCR Only By Specimen Date
Cumulative PCR Tests By Specimen Date

In [35]:
#potentially delete this cell
def wrangle_data(data):
    with open("testingdata.json", "rt") as INFILE:
        data=json.load(INFILE)
    datalist=data["data"]
    #print(datalist)
    dates=[dictionary["date"] for dictionary in datalist]
    dates.sort()
    
    def parse_date(datestring):
        """Convert a date string into a pandas datetime object"""
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq="D")
    testdatadf=pd.DataFrame(index=index, columns=["newcases", "cumcases", "newpcr", "cumpcr"])
    #print(testdatadf)
    for entry in datalist: #each entry is a dictionary with new and cum data for PCR and cases
        date=parse_date(entry["date"])
        for column in ["newcases", "cumcases", "newpcr", "cumpcr"]:
        #first check to make sure there is nothing in there in case data was duplicated
            if pd.isna(testdatadf.loc[date,column]):
                #replace all none with 0 in our data
                value=float(entry[column]) if entry[column]!=None else 0.0
                # .loc is how you access a specific location in the dataframe
                #put index,column in a single set of []
                testdatadf.loc[date, column]=value
    testdatadf.fillna(0.0, inplace=True)
    return testdatadf
            
testdatadf=wrangle_data(data)
print(testdatadf)

           newcases    cumcases  newpcr       cumpcr
2020-01-03      0.0         0.0     0.0         26.0
2020-01-04      0.0         0.0     0.0        161.0
2020-01-05      0.0         0.0     0.0        463.0
2020-01-06      0.0         0.0     0.0        521.0
2020-01-07      0.0         0.0     0.0        659.0
...             ...         ...     ...          ...
2022-10-15   5827.0  20112340.0  1167.0  217735774.0
2022-10-16   7080.0  20119420.0  1128.0  217742588.0
2022-10-17   8634.0  20128054.0  1995.0  217746254.0
2022-10-18   6821.0  20134875.0  1387.0  217762033.0
2022-10-19   4446.0  20139321.0   375.0  217777535.0

[1021 rows x 4 columns]


## Refresh the Data
To refresh the data from PHE press the button below and the graph will update with the new values.

In [38]:
def access_api(button):
    #this code polls the API
    from uk_covid19 import Cov19API
    filters = [
    'areaType=nation',
    'areaName=England'
    ]


    # values here are the names of the PHE metrics
    structure = {
        "date": "date",
        "newcases": "newCasesBySpecimenDate", #new cases by Specimen date
        "cumcases": "cumCasesBySpecimenDate", #cumulative cases by specimen date
        "newpcr": "newCasesPCROnlyBySpecimenDate", #new cases by PCR by specimen date
        "cumpcr": "cumPCRTestsBySpecimenDate" #cumulative cases by PCR by specimen date
    }

    api = Cov19API(filters=filters, structure=structure)
    return {api}

def api_button_callback(button):
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    #testingdata=api.get_json()
    apibutton.icon="check"
    #apibutton.disabled=True
    
apibutton=wdg.Button(
    description="Refresh data",
    disabled=False,
    button_style="success",
    tooltip="Click to download current Public Health England data.",
    icon="download"
)

apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)

Button(button_style='success', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Clic…

In [39]:
def refresh_graph():
    current=graph.value
    if current==graph.options[0]:
        other=graph.options[1]
    else:
        other=graph.options[0]
    graph.value=other
    graph.value=current

## Graph comparing number of Covid cases compared to number of PCR tests

This graph demonstrates the relationship between the number of new Covid 19 cases and the number of PCR tests. This graph also allows the user to compare the cumulative data versus the new data to compare trends in both.

In [40]:
series=wdg.SelectMultiple(
    options=["newcases", "cumcases", "newpcr", "cumpcr"],
    value=["newcases", "cumcases", "newpcr", "cumpcr"],
    rows=4,
    description="Stats:",
    disabled=False
)

scale=wdg.RadioButtons(
    options=["linear", "log"],
    description="Scale:",
    disabled=False
)

controls=wdg.HBox([series, scale])

def testdata_graph(gcols, gscale):
    if gscale=="linear":
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        testdatadf[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(testdata_graph, {"gcols": series, "gscale": scale})

display(controls, graph)

Output()

**Maddy Creach** *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*